In [13]:
# For data processing
import numpy as np
from math import sqrt
# For data processing and manipulation
import pandas as pd
import csv
 
# For date calculations
import datetime
import time
 
# For ploting data
import IPython
import IPython.display
 
import itertools
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
# For checking path
import os , gc
from os import path
import csv
import json
from IPython.display import clear_output 
 
from scipy.stats import hmean
 
from sklearn.metrics import mean_squared_error
import requests

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [14]:
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
'''
Should move db code to other file
'''
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
client = MongoClient(port=27017)
db = client.test

In [3]:
ROOTPATH = 'C:\\Study\\CSE 4000\\New folder\\Thesis\\'
#PATH_IMAGE = ROOTPATH+'images_chicago\\'
#PATH_WEAT_CHI = ROOTPATH+"dataset\\processed_weather_data_chicago_2004_2017.csv"

test query

In [3]:
requestData = {
    'wx_phrase': 'Mostly Cloudy',
    'pressure': 1003.99}
#urlTest = DATABASE_URL + "/query/find/chicago/weather"
#response = requests.get(urlTest, json=requestData)
response = db.weather.chicago.find(requestData)

#do something with res
print(response)
"""
if response.ok:
    try:
        data = response.json()
        print(len(data))
    except json.JSONDecodeError as e:
        print('Error in decoding json: ' + e)
    else:  
        print("Error:", response.status_code)"""

'\nif response.ok:\n    try:\n        data = response.json()\n        print(len(data))\n    except json.JSONDecodeError as e:\n        print(\'Error in decoding json: \' + e)\n    else:  \n        print("Error:", response.status_code)'

In [71]:
del response

<h1> Columns </h1>

In [4]:
# Record the start time
start_time = time.time()

# Get a single document from the collection
sample_document = db.weather.chicago.find_one()

# Extract field names from the document
field_names = list(sample_document.keys())

print("Field Names:", field_names)
print("Number of Fields:", len(field_names))

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))


Field Names: ['_id', '', 'expire_time_gmt', 'valid_time_gmt', 'temp', 'wx_phrase', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 'wdir', 'wspd', 'max_temp', 'min_temp', 'expire_date_time', 'valid_date_time']
Number of Fields: 17
Execution Time: 0.00 seconds


In [73]:
del sample_document, field_names

<h3><b> Phrase Types without Pipeline </h3></b>

In [77]:
# Record the start time
start_time = time.time()

requestData = {
 'field': 'wx_phrase'}

#urlTest = DATABASE_URL + "/query/unique/chicago/weather"
#x = requests.get(urlTest, json=requestData)
distinct_wx_phrases = db.weather.chicago.distinct('wx_phrase')
# Print the distinct values
for phrase in distinct_wx_phrases:
    print(phrase)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

Blowing Snow
Blowing Snow / Windy
Cloudy
Cloudy / Windy
Drizzle
Drizzle and Fog
Fair
Fair / Windy
Fog
Fog / Windy
Funnel Cloud
Haze
Haze / Windy
Heavy Rain
Heavy Rain / Windy
Heavy Snow
Heavy Snow / Windy
Heavy T-Storm
Heavy T-Storm / Windy
Light Drizzle
Light Drizzle / Windy
Light Freezing Drizzle
Light Freezing Rain
Light Rain
Light Rain / Windy
Light Rain with Thunder
Light Sleet
Light Sleet / Windy
Light Snow
Light Snow / Freezing Rain
Light Snow / Windy
Light Snow and Sleet
Light Snow and Sleet / Windy
Mostly Cloudy
Mostly Cloudy / Windy
Partly Cloudy
Partly Cloudy / Windy
Patches of Fog
Rain
Rain / Freezing Rain
Rain / Windy
Rain and Sleet
Rain and Snow
Rain and Snow / Windy
Shallow Fog
Sleet / Windy
Smoke
Snow
Snow / Freezing Rain
Snow / Windy
Snow and Sleet
Snow and Sleet / Windy
Snow and Thunder
T-Storm
T-Storm / Windy
Thunder
Thunder / Windy
Thunder in the Vicinity
Tornado
Widespread Dust / Windy
Wintry Mix
Wintry Mix / Windy
Execution Time: 0.08 seconds


In [80]:
del distinct_wx_phrases

NameError: name 'distinct_wx_phrases' is not defined

<h3><b> Phrase Types using Pipeline </h3></b>

In [8]:
# Record the start time
start_time = time.time()

phraseXCount = {}
#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
                "_id": "$wx_phrase",
                "Count" : {"$sum" : 1}
        }
    }
]
# Run the aggregation query
data_cursor = db.weather.chicago.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Initialize variables
phraseXCount = {}
tsum = sum(doc['Count'] for doc in data)
c = 0

# Process each document
for doc in data:
    print(doc)
    phraseXCount[doc['_id']] = doc['Count']
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total Unique Types =", c)
print("Total Cases =", tsum)
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': 'Heavy Rain / Windy', 'Count': 16}
{'_id': 'Rain / Freezing Rain', 'Count': 4}
{'_id': 'Light Freezing Rain', 'Count': 16}
{'_id': 'Partly Cloudy', 'Count': 18971}
{'_id': 'Snow', 'Count': 184}
{'_id': 'Drizzle', 'Count': 8}
{'_id': 'Heavy Rain', 'Count': 187}
{'_id': 'Cloudy / Windy', 'Count': 3526}
{'_id': 'Light Drizzle / Windy', 'Count': 58}
{'_id': 'Light Drizzle', 'Count': 1439}
{'_id': 'Wintry Mix', 'Count': 375}
{'_id': 'Cloudy', 'Count': 67184}
{'_id': 'Thunder / Windy', 'Count': 27}
{'_id': 'Haze / Windy', 'Count': 6}
{'_id': 'Snow / Windy', 'Count': 22}
{'_id': 'Light Rain with Thunder', 'Count': 779}
{'_id': 'Heavy Snow / Windy', 'Count': 5}
{'_id': 'Light Rain', 'Count': 5197}
{'_id': 'Rain and Snow / Windy', 'Count': 3}
{'_id': 'Tornado', 'Count': 1}
{'_id': 'Patches of Fog', 'Count': 1}
{'_id': 'Snow and Sleet', 'Count': 9}
{'_id': 'Snow and Sleet / Windy', 'Count': 1}
{'_id': 'Light Snow and Sleet / Windy', 'Count': 5}
{'_id': 'Fog', 'Count': 1640}
{'_id': 'Driz

<h3><b> 4.2.5 Average Temprature</h3></b>

In [37]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
"""pipeline = [
    {
        "$group": {
            "_id": "$temp",
            "total_temp": {"$sum": 1},
        }
    },
    {
        "$group": {
            "_id": None,
            "average_temp_per_year": {"$avg": "$total_temp"},
        }
    }
]"""
pipeline = [
    {
        "$group": {
            "_id": None,
            "avg_temperature": { "$avg": "$temp" }
        }
    }
]
# Run the aggregation query
data_cursor = db.weather.chicago.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.3f} seconds".format(execution_time))
        

{'_id': None, 'avg_temperature': 10.897020225413}
Execution Time: 0.082 seconds


<h3><b> 4.2.4 Max Temp Per Year </h3></b>

In [92]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "max_temperature": { "$max": "$temp" }
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.chicago.aggregate(pipeline)

# Convert cursor to list (if dataset is large, consider processing each document in a loop instead)
data = list(data_cursor)

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': None, 'max_temperature': 40.0}
Execution Time: 0.06 seconds


<h3><b> working min Temprature </h3></b>

In [95]:
# Record the start time
start_time = time.time()

#rlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "min_temperature": { "$min": "$temp" }
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.chicago.aggregate(pipeline)

# Process each document in the cursor
for doc in data_cursor:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'_id': None, 'min_temperature': -29.0}
Execution Time: 0.06 seconds


<h3><b>working max temp per year </h3></b>

In [98]:
# Record the start time
start_time = time.time()

#urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"

# Aggregation pipeline
pipeline = [
    {
        "$addFields": {
            "expire_date_time": {
                "$dateFromString": {
                    "dateString": "$expire_date_time",
                    "format": "%d/%m/%Y %H:%M:%S"  # Adjusted date format here
                }
            }
        }
    },
    {
        "$addFields": {
            "year": {"$year": "$expire_date_time"}
        }
    },
    {
        "$group": {
            "_id": "$year",
            "max_temperature": {"$max": "$temp"}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.chicago.aggregate(pipeline)

# Stream process each document in the cursor
for doc in data_cursor:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': 2004, 'max_temperature': 33.0}
{'_id': 2005, 'max_temperature': 40.0}
{'_id': 2006, 'max_temperature': 37.0}
{'_id': 2007, 'max_temperature': 34.0}
{'_id': 2008, 'max_temperature': 34.0}
{'_id': 2009, 'max_temperature': 34.0}
{'_id': 2010, 'max_temperature': 34.0}
{'_id': 2011, 'max_temperature': 38.0}
{'_id': 2012, 'max_temperature': 39.0}
{'_id': 2013, 'max_temperature': 36.0}
{'_id': 2014, 'max_temperature': 34.0}
{'_id': 2015, 'max_temperature': 34.0}
{'_id': 2016, 'max_temperature': 34.0}
{'_id': 2017, 'max_temperature': 35.0}
{'_id': 2018, 'max_temperature': 36.0}
{'_id': 2019, 'max_temperature': 36.0}
{'_id': 2020, 'max_temperature': 35.0}
{'_id': 2021, 'max_temperature': 35.0}
{'_id': 2022, 'max_temperature': 38.0}
Execution Time: 0.25 seconds


<h1><b> 4.1.1 Count Rainy Days </h1></b>

In [102]:
# Record the start time
start_time = time.time()
import re
match_word = "Rain"
# Aggregation pipeline to match wx_phrase with 'Rain' (case insensitive)
pipeline = [
    {
        "$match": {
            "wx_phrase": {"$regex": match_word, "$options": "i"}
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.chicago.aggregate(pipeline)

# Initialize count
c = 0
rainXcount = {}

# Stream process each document in the cursor
for doc in data_cursor:
    rainXcount[doc['_id']] = doc['expire_date_time']
    print(f"On date : {doc['expire_date_time']} , {doc['wx_phrase']} was observed")
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total Cases =", c)
print("Execution Time: {:.2f} seconds".format(execution_time))
#{"wx_phrase": {"$regex": "Rain", "$options": "i"}}
# .113 seconds

On date : 19/03/2014 17:00:00 , Light Rain was observed
On date : 19/03/2014 18:00:00 , Light Rain was observed
On date : 19/03/2014 18:00:00 , Light Rain was observed
On date : 19/03/2014 19:00:00 , Light Rain was observed
On date : 19/03/2014 20:00:00 , Rain was observed
On date : 19/03/2014 20:00:00 , Light Rain was observed
On date : 19/03/2014 20:00:00 , Light Rain was observed
On date : 19/03/2014 21:00:00 , Light Rain was observed
On date : 19/03/2014 22:00:00 , Light Rain was observed
On date : 19/03/2014 23:00:00 , Light Rain was observed
On date : 20/03/2014 00:00:00 , Light Rain was observed
On date : 22/03/2014 09:00:00 , Light Rain was observed
On date : 22/03/2014 10:00:00 , Light Rain was observed
On date : 22/03/2014 11:00:00 , Light Rain was observed
On date : 27/03/2014 17:00:00 , Light Rain was observed
On date : 27/03/2014 18:00:00 , Light Rain was observed
On date : 27/03/2014 18:00:00 , Light Rain was observed
On date : 27/03/2014 19:00:00 , Light Rain was observe

Wind Direction Distribution:



In [105]:
# Record the start time
start_time = time.time()
# Aggregation pipeline for wind direction count
pipeline = [
    {
        "$group": {
            "_id": "$wdir",
            "Intensity": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
]

# Run the aggregation query
data_cursor = db.weather.chicago.aggregate(pipeline)

# Initialize variables
wdirXCount = {}
tsum = 0
c = 0

# Stream process each document in the cursor
for doc in data_cursor:
    print(doc)
    wdirXCount[doc['_id']] = doc['Intensity']
    tsum += doc['Intensity']
    c += 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Print results
print("Total directions =", c)
print("Total Cases =", tsum)
print("Execution Time: {:.2f} seconds".format(execution_time))

{'_id': None, 'Intensity': 12837}
{'_id': 10.0, 'Intensity': 4672}
{'_id': 20.0, 'Intensity': 5010}
{'_id': 30.0, 'Intensity': 5464}
{'_id': 40.0, 'Intensity': 4193}
{'_id': 50.0, 'Intensity': 3804}
{'_id': 60.0, 'Intensity': 4277}
{'_id': 70.0, 'Intensity': 4188}
{'_id': 80.0, 'Intensity': 4648}
{'_id': 90.0, 'Intensity': 4798}
{'_id': 100.0, 'Intensity': 4307}
{'_id': 110.0, 'Intensity': 3142}
{'_id': 120.0, 'Intensity': 2851}
{'_id': 130.0, 'Intensity': 2748}
{'_id': 140.0, 'Intensity': 3199}
{'_id': 150.0, 'Intensity': 2868}
{'_id': 160.0, 'Intensity': 3678}
{'_id': 170.0, 'Intensity': 5155}
{'_id': 180.0, 'Intensity': 6148}
{'_id': 190.0, 'Intensity': 7328}
{'_id': 200.0, 'Intensity': 9216}
{'_id': 210.0, 'Intensity': 8795}
{'_id': 220.0, 'Intensity': 6438}
{'_id': 230.0, 'Intensity': 5725}
{'_id': 240.0, 'Intensity': 6660}
{'_id': 250.0, 'Intensity': 6774}
{'_id': 260.0, 'Intensity': 6079}
{'_id': 270.0, 'Intensity': 6832}
{'_id': 280.0, 'Intensity': 5968}
{'_id': 290.0, 'Intensi

<h1><b> 4.1.2 In A Range Check</h1></b>


In [5]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$match": {
            "$and": [
                {"pressure": {"$gt": 1000.00}},
                {"pressure": {"$lt": 1001.00}}
            ]
        }
    }
]

# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")

,_id,,expire_time_gmt,valid_time_gmt,temp,wx_phrase,heat_index,rh,pressure,vis,wc,wdir,wspd,expire_date_time,valid_date_time,max_temp,min_temp
0,65b746bfcf903dd4f8403abc,11,1072983180,1072975980,3.0,Cloudy,3.0,65.0,1000.08,16.0,-1.0,170.0,19.0,02/01/2004 01:00:00,01/01/2004 23:00:00,NaN,NaN
1,65b746bfcf903dd4f8403b44,147,1073325180,1073317980,-5.0,Mostly Cloudy,-5.0,81.0,1000.96,16.0,-12.0,330.0,24.0,06/01/2004 00:00:00,05/01/2004 22:00:00,NaN,NaN
2,65b746bfcf903dd4f8403b46,149,1073332380,1073325180,-4.0,Cloudy,-4.0,78.0,1000.76,16.0,-11.0,330.0,22.0,06/01/2004 02:00:00,06/01/2004 00:00:00,-3.0,-5.0
3,65b746bfcf903dd4f8403b47,150,1073335980,1073328780,-4.0,Mostly Cloudy,-4.0,75.0,1000.66,16.0,-10.0,320.0,19.0,06/01/2004 03:00:00,06/01/2004 01:00:00,NaN,NaN
4,65b746bfcf903dd4f8403b48,151,1073339580,1073332380,-3.0,Cloudy,-3.0,69.0,1000.76,16.0,-10.0,310.0,22.0,06/01/2004 04:00:00,06/01/2004 02:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,65b746c9cf903dd4f84330bb,194058,1671673980,1671666780,-1.0,Cloudy,-1.0,76.0,1000.30,11.0,-5.0,130.0,13.0,22/12/2022 08:00:00,22/12/2022 06:00:00,NaN,NaN
5729,65b746c9cf903dd4f84330bc,194059,1671677580,1671670380,0.0,Haze,0.0,75.0,1000.30,9.0,-3.0,140.0,7.0,22/12/2022 09:00:00,22/12/2022 07:00:00,NaN,NaN
5730,65b746c9cf903dd4f8433160,194223,1672105980,1672098780,-6.0,Mostly Cloudy,-6.0,68.0,1000.63,16.0,-11.0,310.0,11.0,27/12/2022 08:00:00,27/12/2022 06:00:00,NaN,NaN
5731,65b746c9cf903dd4f843316e,194237,1672152780,1672145580,-13.0,Partly Cloudy,-13.0,80.0,1000.96,16.0,-19.0,210.0,11.0,27/12/2022 21:00:00,27/12/2022 19:00:00,NaN,NaN


Execution Time: 0.31 seconds


<h1><b> Complex Queries </h1></b>

<h3> total sum of temperatures and pressures for each distinct weather phrase (wx_phrase) </h3>

In [7]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$group": {
            "_id": "$wx_phrase",
            "total_temp": {"$sum": "$temp"},
            "total_pressure": {"$sum": "$pressure"}
        }
    },
    {
        "$sort": {"_id": 1}
    },
    {
        "$project": {
            "_id": 0,
            "wx_phrase": "$_id",
            "total_temp": 1,
            "total_pressure": 1
        }
    }
]

# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#.186 seconds

,total_temp,total_pressure,wx_phrase
0,34.0,3976.91,None
1,-215.0,10876.14,Blowing Snow
2,-632.0,33614.80,Blowing Snow / Windy
3,567051.0,66652048.02,Cloudy
4,21287.0,3477722.62,Cloudy / Windy
...,...,...,...
58,1401.0,55414.06,Thunder in the Vicinity
59,24.0,989.04,Tornado
60,28.0,980.10,Widespread Dust / Windy
61,157.0,371247.08,Wintry Mix


Execution Time: 0.13 seconds


<h3> Daily Data</h3>

In [19]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "day": {
                "$dateFromString": {
                    "dateString": "$valid_date_time",
                    "format": "%d/%m/%Y %H:%M:%S"  # Include time in the format
                }
            }
        }
    },
    {
        "$group": {
            "_id": {
                "day": {
                    "$dateToString": { "format": "%Y-%m-%d", "date": "$day" }
                }
            },
            "avg_temp": {"$avg": "$temp"},
            "total_wspd": {"$sum": "$wspd"},
            "max_visibility": {"$max": "$vis"},
            "distinct_weather_conditions": {"$addToSet": "$wx_phrase"}
        }
    },
    {
        "$addFields": {
            "count_distinct_weather_conditions": {"$size": "$distinct_weather_conditions"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "day": "$_id.day",
            "avg_temp": 1,
            "total_wspd": 1,
            "max_visibility": 1,
            "distinct_weather_conditions": "$count_distinct_weather_conditions"
        }
    },
    {
        "$sort": {"day": 1}
    }
]


# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.13 seconds

,avg_temp,total_wspd,max_visibility,day,distinct_weather_conditions
0,0.000000,125.0,16.0,2004-01-01,2
1,7.027778,489.0,16.0,2004-01-02,3
2,11.516129,651.0,16.0,2004-01-03,6
3,1.000000,742.0,16.0,2004-01-04,4
4,-1.852941,875.0,16.0,2004-01-05,3
...,...,...,...,...,...
6935,-9.000000,279.0,16.0,2022-12-27,5
6936,-3.320000,659.0,16.0,2022-12-28,2
6937,7.541667,690.0,16.0,2022-12-29,6
6938,11.592593,655.0,16.0,2022-12-30,6


Execution Time: 0.53 seconds


<h1> <b> addition </b></h1>

<h1> <b> subtraction </b></h1>

<h1> <b> comparing </b></h1>

<h3> 4.1.3 Selecting records with temperature differences within a specified range and wind speed above a certain threshold </h3>

In [5]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "temperature_difference": {
                "$subtract": ["$max_temp", "$min_temp"]
            },
            "wind_speed_category": {
                "$cond": {
                    "if": { "$gt": ["$wspd", 20] },
                    "then": "High wind",
                    "else": "Low wind"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "expire_time_gmt": 1,
            "valid_time_gmt": 1,
            "temp": 1,
            "wspd": 1,
            "wx_phrase": 1,
            "valid_date_time": 1,
            "expire_date_time": 1,
            "temperature_difference": 1,
            "wind_speed_category": 1
        }
    }
]

# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.144 seconds

,expire_time_gmt,valid_time_gmt,temp,wx_phrase,wspd,expire_date_time,valid_date_time,temperature_difference,wind_speed_category
0,1072943580,1072936380,-1.0,Mostly Cloudy,6.0,01/01/2004 14:00:00,01/01/2004 12:00:00,4.0,Low wind
1,1072947180,1072939980,-1.0,Mostly Cloudy,9.0,01/01/2004 15:00:00,01/01/2004 13:00:00,NaN,Low wind
2,1072950780,1072943580,-1.0,Mostly Cloudy,NaN,01/01/2004 16:00:00,01/01/2004 14:00:00,NaN,Low wind
3,1072954380,1072947180,-1.0,Mostly Cloudy,7.0,01/01/2004 17:00:00,01/01/2004 15:00:00,NaN,Low wind
4,1072957980,1072950780,-1.0,Mostly Cloudy,6.0,01/01/2004 18:00:00,01/01/2004 16:00:00,NaN,Low wind
...,...,...,...,...,...,...,...,...,...
194326,1672458780,1672451580,2.0,Cloudy,6.0,31/12/2022 10:00:00,31/12/2022 08:00:00,NaN,Low wind
194327,1672462380,1672455180,2.0,Mostly Cloudy,7.0,31/12/2022 11:00:00,31/12/2022 09:00:00,NaN,Low wind
194328,1672465980,1672458780,1.0,Partly Cloudy,11.0,31/12/2022 12:00:00,31/12/2022 10:00:00,NaN,Low wind
194329,1672469580,1672462380,1.0,Mostly Cloudy,11.0,31/12/2022 13:00:00,31/12/2022 11:00:00,NaN,Low wind


Execution Time: 1.01 seconds


<h3> 4.1.4 Identifying records where visibility is within a certain range and the temperature is below freezing </h3>

In [5]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline =[
    {
        "$addFields": {
            "wind_direction": {
                "$cond": {
                    "if": {
                        "$and": [
                            {"$gte": ["$wdir", 0]},
                            {"$lte": ["$wdir", 180]}
                        ]
                    },
                    "then": "Easterly Wind",
                    "else": "Westerly Wind"
                }
            },
            "temperature_status": {
                "$cond": {
                    "if": {"$lt": ["$temp", 0]},
                    "then": "Below freezing",
                    "else": "Above freezing"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "expire_date_time": 1,
            "expire_time_gmt": 1,
            "valid_date_time": 1,
            "valid_time_gmt": 1,
            "vis": 1,
            "wx_phrase": 1,
            "wind_direction": 1,
            "temp": 1,
            "temperature_status": 1
        }
    }
]



# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.137 seconds

#.212 seconds
#project
"""{
           "_id": 0,
            "Unnamed: 0": 1,
            "expire_date_time": 1,
            "expire_time_gmt": 1,
            "valid_date_time": 1,
            "valid_time_gmt": 1,
            "vis": 1,
            "wx_phrase": 1,
"wind_direction": {
  "$cond": {
    "if": { "$and": [ { "$gte": [ "$wdir", 0 ] },{ "$lte": [ "$wdir", 180 ] } ] },
    "then": "Easterly Wind",
    "else": "Westerly Wind"
   }
 },
 "temp": 1,
 "temperature_status": {
   "$cond": {
     "if": { "$lt": [ "$temp", 0 ] },
     "then": "Below freezing",
     "else": "Above freezing"
    }
  }
}"""

,expire_time_gmt,valid_time_gmt,temp,wx_phrase,vis,expire_date_time,valid_date_time,wind_direction,temperature_status
0,1072943580,1072936380,-1.0,Mostly Cloudy,16.0,01/01/2004 14:00:00,01/01/2004 12:00:00,Easterly Wind,Below freezing
1,1072947180,1072939980,-1.0,Mostly Cloudy,16.0,01/01/2004 15:00:00,01/01/2004 13:00:00,Easterly Wind,Below freezing
2,1072950780,1072943580,-1.0,Mostly Cloudy,16.0,01/01/2004 16:00:00,01/01/2004 14:00:00,Westerly Wind,Below freezing
3,1072954380,1072947180,-1.0,Mostly Cloudy,16.0,01/01/2004 17:00:00,01/01/2004 15:00:00,Easterly Wind,Below freezing
4,1072957980,1072950780,-1.0,Mostly Cloudy,16.0,01/01/2004 18:00:00,01/01/2004 16:00:00,Easterly Wind,Below freezing
...,...,...,...,...,...,...,...,...,...
194326,1672458780,1672451580,2.0,Cloudy,14.0,31/12/2022 10:00:00,31/12/2022 08:00:00,Westerly Wind,Above freezing
194327,1672462380,1672455180,2.0,Mostly Cloudy,14.0,31/12/2022 11:00:00,31/12/2022 09:00:00,Westerly Wind,Above freezing
194328,1672465980,1672458780,1.0,Partly Cloudy,16.0,31/12/2022 12:00:00,31/12/2022 10:00:00,Westerly Wind,Above freezing
194329,1672469580,1672462380,1.0,Mostly Cloudy,16.0,31/12/2022 13:00:00,31/12/2022 11:00:00,Westerly Wind,Above freezing


Execution Time: 1.00 seconds


'{\n           "_id": 0,\n            "Unnamed: 0": 1,\n            "expire_date_time": 1,\n            "expire_time_gmt": 1,\n            "valid_date_time": 1,\n            "valid_time_gmt": 1,\n            "vis": 1,\n            "wx_phrase": 1,\n"wind_direction": {\n  "$cond": {\n    "if": { "$and": [ { "$gte": [ "$wdir", 0 ] },{ "$lte": [ "$wdir", 180 ] } ] },\n    "then": "Easterly Wind",\n    "else": "Westerly Wind"\n   }\n },\n "temp": 1,\n "temperature_status": {\n   "$cond": {\n     "if": { "$lt": [ "$temp", 0 ] },\n     "then": "Below freezing",\n     "else": "Above freezing"\n    }\n  }\n}'

<h3> 4.1.5 Comparing temperature differences with visibility ranges to classify weather severity</h3>

In [10]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "realfeel_difference": {"$subtract": ["$temp", "$wc"]},
            "visibility_status": {
                "$switch": {
                    "branches": [
                        {"case": {"$lt": ["$vis", 5]}, "then": "Low Visibility"},
                        {"case": {"$and": [{"$gte": ["$vis", 5]}, {"$lte": ["$vis", 15]}]}, "then": "Moderate Visibility"}
                    ],
                    "default": "High Visibility"
                }
            },
            "wind_chill_effect": {
                "$cond": {
                    "if": {"$gt": [{"$subtract": ["$temp", "$wc"]}, 20]},
                    "then": "Significant Wind Chill Effect",
                    "else": "Negligible Wind Chill Effect"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "valid_time_gmt": 1,
            "realfeel_difference": 1,
            "vis": 1,
            "visibility_status": 1,
            "wind_chill_effect": 1,
            "wx_phrase": 1,
            "valid_date_time": 1
        }
    }
]

# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.139 seconds

#{"_id": 0,"wind_chill_effect": {"$cond": [{"$gt": [{"$subtract": ["$temp", "$wc"]}, 20]}, "Significant Wind Chill Effect", "Negligible Wind Chill Effect"]}, "wx_phrase": 1, "valid_date_time": 1, "Unnamed: 0": 1, "valid_time_gmt": 1, "temp": 1, "vis": 1, "realfeel_difference": {"$subtract": ["$temp", "$wc"]}, "visibility_status": {"$cond": [{"$lt": ["$vis", 5]}, "Low Visibility", {"$cond": [{"$and": [{"$gte": ["$vis", 5]}, {"$lte": ["$vis", 15]}]}, "Moderate Visibility", "High Visibility"]}]},"wind_chill_effect":{"$cond":{"if":{"$gt":[{"$subtract":["$temp","$wc"]},20]},"then":"Significant Wind Chill Effect","else":"Negligible Wind Chill Effect"}},"wx_phrase":1,"valid_date_time":1 }

,valid_time_gmt,wx_phrase,vis,valid_date_time,realfeel_difference,visibility_status,wind_chill_effect
0,1072936380,Mostly Cloudy,16.0,01/01/2004 12:00:00,0.0,High Visibility,Negligible Wind Chill Effect
1,1072939980,Mostly Cloudy,16.0,01/01/2004 13:00:00,3.0,High Visibility,Negligible Wind Chill Effect
2,1072943580,Mostly Cloudy,16.0,01/01/2004 14:00:00,0.0,High Visibility,Negligible Wind Chill Effect
3,1072947180,Mostly Cloudy,16.0,01/01/2004 15:00:00,2.0,High Visibility,Negligible Wind Chill Effect
4,1072950780,Mostly Cloudy,16.0,01/01/2004 16:00:00,0.0,High Visibility,Negligible Wind Chill Effect
...,...,...,...,...,...,...,...
194326,1672451580,Cloudy,14.0,31/12/2022 08:00:00,0.0,Moderate Visibility,Negligible Wind Chill Effect
194327,1672455180,Mostly Cloudy,14.0,31/12/2022 09:00:00,3.0,Moderate Visibility,Negligible Wind Chill Effect
194328,1672458780,Partly Cloudy,16.0,31/12/2022 10:00:00,4.0,High Visibility,Negligible Wind Chill Effect
194329,1672462380,Mostly Cloudy,16.0,31/12/2022 11:00:00,4.0,High Visibility,Negligible Wind Chill Effect


Execution Time: 0.99 seconds


<h3> 4.1.6 Calculating wind direction variance and assigning weather status based on wind chill </h3>

In [11]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
    {
        "$addFields": {
            "opposite_wdir": {"$add": ["$wdir", 180]},
            "reduced_wind_chill": {"$subtract": ["$wc", 5]},
            "wind_temperature_factor": {"$multiply": ["$temp", "$wspd"]},
            "comfort_index": {"$divide": [{"$add": ["$temp", "$wspd"]}, 2]},
            "comfort_status": {
                "$switch": {
                    "branches": [
                        {"case": {"$gt": ["$temp", 85]}, "then": "Too Hot"},
                        {"case": {"$lt": ["$temp", 32]}, "then": "Too Cold"}
                    ],
                    "default": "Comfortable"
                }
            },
            "wind_chill_status": {
                "$switch": {
                    "branches": [
                        {"case": {"$lt": ["$wc", 0]}, "then": "Extreme Cold"},
                        {"case": {"$and": [{"$gte": ["$wc", 0]}, {"$lte": ["$wc", 32]}]}, "then": "Chilly"}
                    ],
                    "default": "Not Chilly"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "Unnamed: 0": 1,
            "valid_time_gmt": 1,
            "opposite_wdir": 1,
            "reduced_wind_chill": 1,
            "wind_temperature_factor": 1,
            "comfort_index": 1,
            "comfort_status": 1,
            "wind_chill_status": 1,
            "wx_phrase": 1,
            "valid_date_time": 1
        }
    }
]
# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")
#0.142 seconds

,valid_time_gmt,wx_phrase,valid_date_time,opposite_wdir,reduced_wind_chill,wind_temperature_factor,comfort_index,comfort_status,wind_chill_status
0,1072936380,Mostly Cloudy,01/01/2004 12:00:00,190.0,-6.0,-6.0,2.5,Too Cold,Extreme Cold
1,1072939980,Mostly Cloudy,01/01/2004 13:00:00,200.0,-9.0,-9.0,4.0,Too Cold,Extreme Cold
2,1072943580,Mostly Cloudy,01/01/2004 14:00:00,NaN,-6.0,NaN,NaN,Too Cold,Extreme Cold
3,1072947180,Mostly Cloudy,01/01/2004 15:00:00,270.0,-8.0,-7.0,3.0,Too Cold,Extreme Cold
4,1072950780,Mostly Cloudy,01/01/2004 16:00:00,290.0,-6.0,-6.0,2.5,Too Cold,Extreme Cold
...,...,...,...,...,...,...,...,...,...
194326,1672451580,Cloudy,31/12/2022 08:00:00,440.0,-3.0,12.0,4.0,Too Cold,Chilly
194327,1672455180,Mostly Cloudy,31/12/2022 09:00:00,480.0,-6.0,14.0,4.5,Too Cold,Extreme Cold
194328,1672458780,Partly Cloudy,31/12/2022 10:00:00,480.0,-8.0,11.0,6.0,Too Cold,Extreme Cold
194329,1672462380,Mostly Cloudy,31/12/2022 11:00:00,480.0,-8.0,11.0,6.0,Too Cold,Extreme Cold


Execution Time: 1.18 seconds


<h2> 4.2.1 COUNT </h2>

In [23]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
  {
    "$group": {
      "_id": None, # Grouping without a specific field to aggregate over the entire collection
      "Total_Observations": { "$sum": 1 },
      "Mostly_Cloudy_Count": {
        "$sum": {
          "$cond": [{ "$eq": ["$wx_phrase", "Mostly Cloudy"] }, 1, 0]
        }
      },
      "Below_Zero_Count": {
        "$sum": {
          "$cond": [{ "$lt": ["$temp", 0] }, 1, 0]
        }
      },
      "Windy_Conditions_Count": {
        "$sum": {
          "$cond": [{ "$gte": ["$wspd", 10] }, 1, 0]
        }
      },
      "Good_Visibility_Count": {
        "$sum": {
          "$cond": [{ "$gt": ["$vis", 10] }, 1, 0]
        }
      }
    }
  }
]

# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.2f} seconds")

,_id,Total_Observations,Mostly_Cloudy_Count,Below_Zero_Count,Windy_Conditions_Count,Good_Visibility_Count
0,None,194331,41577,35363,145762,158749


Execution Time: 0.16 seconds


<h2> 4.2.2 SUM </h2>

In [31]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
  {
    "$group": {
      "_id": None,
      "Total_Wind_Speed": { "$sum": "$wspd" },
      "Sum_Wind_Speed_Below_Zero": { 
        "$sum": { 
          "$cond": [
            { "$lt": ["$temp", 0] },
            "$wspd",
            0
          ]
        }
      }
    }
  }
]

# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.3f} seconds")

,_id,Total_Wind_Speed,Sum_Wind_Speed_Below_Zero
0,None,3122962.0,607689.0


Execution Time: 0.108 seconds


<h1> 4.2.3 MIN </h1>

In [32]:
# Record the start time
start_time = time.time()

# Aggregation pipeline to match documents with pressure between 1000.00 and 1001.00
pipeline = [
  {
    "$group": {
      "_id": None,
      "Min_Wind_Speed": { "$min": "$wspd" },
      "Min_Visibility": { "$min": "$vis" }
    }
  }
]

# Run the aggregation query
data = db.weather.chicago.aggregate(pipeline)

# Collect the results into a list of dictionaries
results = list(data)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Print the DataFrame as a table
display(df)  # Use print(df) if not in a Jupyter notebook

# Print execution time
print(f"Execution Time: {execution_time:.3f} seconds")

,_id,Min_Wind_Speed,Min_Visibility
0,None,0.0,0.0


Execution Time: 0.112 seconds
